In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_unixtime
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer, IndexToString
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from pyspark.sql.functions import col, sum as _sum

## Data Extraction from HDFS with PySpark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SteamReviewsHDFS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SteamReviewsHDFS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "100")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 13:05:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/28 13:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/28 13:05:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
df = spark.read.csv("/user/tejashree/project/steam_reviews.csv", header=True, inferSchema=True)


In [6]:
df.show(5)


+---+------+--------------------+---------+--------+----------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|_c0|app_id|            app_name|review_id|language|                            review|timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author.steamid|author.num_games_owned|author.num_reviews|author.playtime_forever|author.playtime_last_two_weeks|author.playtime_at_review|author.last_played|
+---+------+--------------------+---------+--------+----------------------------------+-----------------+-----------------+-----------+-------------+-----------+-----

25/04/27 22:42:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user/tejashree/project/steam_reviews.csv


In [7]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: string (nullable = true)
 |-- timestamp_updated: string (nullable = true)
 |-- recommended: string (nullable = true)
 |-- votes_helpful: string (nullable = true)
 |-- votes_funny: string (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- steam_purchase: string (nullable = true)
 |-- received_for_free: string (nullable = true)
 |-- written_during_early_access: string (nullable = true)
 |-- author.steamid: string (nullable = true)
 |-- author.num_games_owned: string (nullable = true)
 |-- author.num_reviews: string (nullable = true)
 |-- author.playtime_forever: string (nullable = true)
 |-- author.playtime_last_two_weeks: string (nullable = true)
 |-- author.playt

In [8]:
df.count()

40848659

## Data Cleaning and Pre-Processing

- Convert the column names into valid names - remove the dots and replace them with underscore.
- Remove the NULL records from the data set 
- Convert the columns into appropriate data types. For e.g., app_id can be converted to IntegerType() and all the timestamp strings into timestamps.

In [10]:
data_csv = df.withColumnRenamed("author.steamid","author_steamid"). \
withColumnRenamed("author.num_games_owned","author_num_games_owned").\
withColumnRenamed("author.num_reviews", "author_num_reviews").\
withColumnRenamed("author.playtime_forever","author_playtime_forever").\
withColumnRenamed("author.playtime_at_review","author_playtime_at_review").\
withColumnRenamed("author.last_played","author_last_played").\
withColumnRenamed("author.playtime_last_two_weeks", "author_playtime_last_two_weeks")

In [11]:
data_csv.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: string (nullable = true)
 |-- timestamp_updated: string (nullable = true)
 |-- recommended: string (nullable = true)
 |-- votes_helpful: string (nullable = true)
 |-- votes_funny: string (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- steam_purchase: string (nullable = true)
 |-- received_for_free: string (nullable = true)
 |-- written_during_early_access: string (nullable = true)
 |-- author_steamid: string (nullable = true)
 |-- author_num_games_owned: string (nullable = true)
 |-- author_num_reviews: string (nullable = true)
 |-- author_playtime_forever: string (nullable = true)
 |-- author_playtime_last_two_weeks: string (nullable = true)
 |-- author_playt

## Check NUll

In [13]:
columns_to_check = [c for c in data_csv.columns if c != '_c0']

null_counts = data_csv.select([
    _sum(col(c).isNull().cast("int")).alias(c) for c in columns_to_check
])

In [14]:
null_counts_pd = null_counts.toPandas()
print(null_counts_pd.T)  # T = transpose


                                       0
app_id                          11335874
app_name                        13077913
review_id                       13975798
language                        14498420
review                          14860610
timestamp_created               19928219
timestamp_updated               20104213
recommended                     20217490
votes_helpful                   20296075
votes_funny                     20350386
weighted_vote_score             20389679
comment_count                   20417234
steam_purchase                  20437907
received_for_free               20452713
written_during_early_access     20464051
author_steamid                  20476095
author_num_games_owned          20480003
author_num_reviews              22945129
author_playtime_forever         23511575
author_playtime_last_two_weeks  23759374
author_playtime_at_review       23902511
author_last_played              23951902


## Handling missing values
➡ Drop rows missing app_id, review_id, review, recommended (core fields)
➡ Fill numeric columns like votes_helpful, votes_funny with 0
➡ Ignore user info if missing (optional for dashboard)



In [16]:
important_cols = ["app_id", "review_id", "review", "recommended"]
numeric_fill_cols = ["votes_helpful", "votes_funny", "comment_count", "weighted_vote_score"]

In [17]:
#Drop rows where any important column is NULL
df_clean = data_csv.dropna(subset=important_cols)

#Fill numeric columns (votes helpful, funny, comment_count) with 0 where NULL
for col_name in numeric_fill_cols:
    df_clean = df_clean.withColumn(col_name, coalesce(col_name, lit(0)))

#fill steam_purchase, received_for_free to 'Unknown'
df_clean = df_clean.fillna({
    "steam_purchase": "unknown",
    "received_for_free": "unknown"
})

### Normalize text

In [19]:
# 2. Normalize text:
#    a. Remove HTML tags
#    b. Remove emojis and non-text symbols
#    c. Keep Unicode letters (all languages) and spaces
df_clean = df_clean.withColumn(
    "cleaned_review",
    regexp_replace(
        lower(col("review")),
        "<[^>]*>",  # Remove HTML
        ""
    )
)
df_clean = df_clean.withColumn(
    "cleaned_review",
    regexp_replace(
        col("cleaned_review"),
        "[^\\p{L}\\s]",  # Keep Unicode letters and whitespace only
        ""
    )
)
df_clean = df_clean.withColumn("cleaned_review", trim(col("cleaned_review")))


In [20]:
df_sample = df_clean.limit(10).toPandas()
print(df_sample)

25/04/27 22:43:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user/tejashree/project/steam_reviews.csv


  _c0  app_id                  app_name review_id  language  \
0   0  292030  The Witcher 3: Wild Hunt  85185598  schinese   
1   1  292030  The Witcher 3: Wild Hunt  85185250  schinese   
2   2  292030  The Witcher 3: Wild Hunt  85185111  schinese   
3   3  292030  The Witcher 3: Wild Hunt  85184605   english   
4   4  292030  The Witcher 3: Wild Hunt  85184287  schinese   
5   5  292030  The Witcher 3: Wild Hunt  85184171   english   
6   6  292030  The Witcher 3: Wild Hunt  85184064   english   
7   8  292030  The Witcher 3: Wild Hunt  85183227  schinese   
8   9  292030  The Witcher 3: Wild Hunt  85182785   spanish   
9  10  292030  The Witcher 3: Wild Hunt  85182697  schinese   

                                              review timestamp_created  \
0                            不玩此生遗憾，RPG游戏里的天花板，太吸引人了        1611381629   
1                                 拔DIAO无情打桩机--杰洛特!!!        1611381030   
2                                              巫师3NB        1611380800   
3  One of 

In [21]:
df_clean.count()

20595477

In [22]:
df_clean.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: string (nullable = true)
 |-- timestamp_updated: string (nullable = true)
 |-- recommended: string (nullable = true)
 |-- votes_helpful: string (nullable = false)
 |-- votes_funny: string (nullable = false)
 |-- weighted_vote_score: string (nullable = false)
 |-- comment_count: string (nullable = false)
 |-- steam_purchase: string (nullable = false)
 |-- received_for_free: string (nullable = false)
 |-- written_during_early_access: string (nullable = true)
 |-- author_steamid: string (nullable = true)
 |-- author_num_games_owned: string (nullable = true)
 |-- author_num_reviews: string (nullable = true)
 |-- author_playtime_forever: string (nullable = true)
 |-- author_playtime_last_two_weeks: string (nullable = true)
 |-- author

In [23]:
# Changing the data type of each columns into appropriate types
df_clean = df_clean.withColumn("app_id",df_clean["app_id"].cast(IntegerType())).\
            withColumn("review_id", df_clean["review_id"].cast(LongType())).\
            withColumn("author_steamid", df_clean["author_steamid"].cast(LongType())).\
            withColumn("recommended", df_clean["recommended"].cast(BooleanType())).\
            withColumn("steam_purchase", df_clean["steam_purchase"].cast(BooleanType())).\
            withColumn("author_num_games_owned", df_clean["author_num_games_owned"].cast(IntegerType())).\
            withColumn("author_num_reviews", df_clean["author_num_reviews"].cast(IntegerType())).\
            withColumn("author_playtime_forever", df_clean["author_playtime_forever"].cast(FloatType())).\
            withColumn("author_playtime_at_review", df_clean["author_playtime_at_review"].cast(FloatType()))

In [24]:
# Converting the time columns into timestamp data type
df_clean = df_clean.withColumn("timestamp_created", from_unixtime("timestamp_created").cast("timestamp")).\
            withColumn("author_last_played", from_unixtime(df_clean["author_last_played"]).cast(TimestampType())).\
            withColumn("timestamp_updated", from_unixtime(df_clean["timestamp_updated"]).cast(TimestampType()))

In [25]:
df_clean.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: long (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: timestamp (nullable = true)
 |-- timestamp_updated: timestamp (nullable = true)
 |-- recommended: boolean (nullable = true)
 |-- votes_helpful: string (nullable = false)
 |-- votes_funny: string (nullable = false)
 |-- weighted_vote_score: string (nullable = false)
 |-- comment_count: string (nullable = false)
 |-- steam_purchase: boolean (nullable = true)
 |-- received_for_free: string (nullable = false)
 |-- written_during_early_access: string (nullable = true)
 |-- author_steamid: long (nullable = true)
 |-- author_num_games_owned: integer (nullable = true)
 |-- author_num_reviews: integer (nullable = true)
 |-- author_playtime_forever: float (nullable = true)
 |-- author_playtime_last_two_weeks: string (nullable = true)
 |-- a

In [26]:
df_sample2 = df_clean.limit(10).toPandas()
print(df_sample2)

25/04/27 22:43:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user/tejashree/project/steam_reviews.csv


  _c0  app_id                  app_name  review_id  language  \
0   0  292030  The Witcher 3: Wild Hunt   85185598  schinese   
1   1  292030  The Witcher 3: Wild Hunt   85185250  schinese   
2   2  292030  The Witcher 3: Wild Hunt   85185111  schinese   
3   3  292030  The Witcher 3: Wild Hunt   85184605   english   
4   4  292030  The Witcher 3: Wild Hunt   85184287  schinese   
5   5  292030  The Witcher 3: Wild Hunt   85184171   english   
6   6  292030  The Witcher 3: Wild Hunt   85184064   english   
7   8  292030  The Witcher 3: Wild Hunt   85183227  schinese   
8   9  292030  The Witcher 3: Wild Hunt   85182785   spanish   
9  10  292030  The Witcher 3: Wild Hunt   85182697  schinese   

                                              review   timestamp_created  \
0                            不玩此生遗憾，RPG游戏里的天花板，太吸引人了 2021-01-22 22:00:29   
1                                 拔DIAO无情打桩机--杰洛特!!! 2021-01-22 21:50:30   
2                                              巫师3NB 2021-01-22 21:

## Save the cleaned data to HDFS in Parquet format 

In [28]:
#df_clean.write.mode("overwrite").parquet("/user/tejashree/project/cleaned_steam_reviews.parquet")

25/04/27 22:43:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
 Schema: _c0, app_id, app_name, review_id, language, review, timestamp_created, timestamp_updated, recommended, votes_helpful, votes_funny, weighted_vote_score, comment_count, steam_purchase, received_for_free, written_during_early_access, author.steamid, author.num_games_owned, author.num_reviews, author.playtime_forever, author.playtime_last_two_weeks, author.playtime_at_review, author.last_played
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user/tejashree/project/steam_reviews.csv


## # Changing the data type of each columns into appropriate types

### Data Preprocessing and grouping

In [31]:
# We form a list of columns to be included in different smaller data frames
col_demo = ["app_id", "app_name", "review_id", "language", "author_steamid", "timestamp_created" ,"author_playtime_forever","recommended"]
col_author = ["steam_purchase", 'author_steamid', "author_num_games_owned", "author_num_reviews", "author_playtime_forever", "author_playtime_at_review", "author_last_played","recommended"]
col_time = [ "app_id", "app_name", "timestamp_created", "timestamp_updated", 'author_playtime_at_review', "recommended"]
col_rev = [ "app_id", "app_name", "language", "recommended"]
col_rec = ["app_id", "app_name", "recommended"] # checking recommendation % for a specific game

In [32]:
data_demo = df_clean.select(*col_demo)

In [33]:
data_demo.show()

+----------+--------------------+---------+--------+-----------------+-------------------+-----------------------+-----------+
|    app_id|            app_name|review_id|language|   author_steamid|  timestamp_created|author_playtime_forever|recommended|
+----------+--------------------+---------+--------+-----------------+-------------------+-----------------------+-----------+
|    292030|The Witcher 3: Wi...| 85185598|schinese|76561199095369542|2021-01-22 22:00:29|                 1909.0|       true|
|    292030|The Witcher 3: Wi...| 85185250|schinese|76561198949504115|2021-01-22 21:50:30|                 2764.0|       true|
|    292030|The Witcher 3: Wi...| 85185111|schinese|76561199090098988|2021-01-22 21:46:40|                 1061.0|       true|
|    292030|The Witcher 3: Wi...| 85184605| english|76561199054755373|2021-01-22 21:32:50|                 5587.0|       true|
|    292030|The Witcher 3: Wi...| 85184287|schinese|76561199028326951|2021-01-22 21:23:47|                  217

In [34]:
data_author = df_clean.select(*col_author)
data_author.show()

+--------------+-----------------+----------------------+------------------+-----------------------+-------------------------+-------------------+-----------+
|steam_purchase|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_at_review| author_last_played|recommended|
+--------------+-----------------+----------------------+------------------+-----------------------+-------------------------+-------------------+-----------+
|          true|76561199095369542|                     6|                 2|                 1909.0|                   1909.0|2021-01-22 11:23:03|       true|
|          true|76561198949504115|                    30|                10|                 2764.0|                   2674.0|2021-01-22 23:18:27|       true|
|          true|76561199090098988|                     5|                 1|                 1061.0|                   1060.0|2021-01-22 22:36:17|       true|
|          true|76561199054755373|            

In [35]:
data_time = df_clean.select(*col_time)
data_time.show()

+----------+--------------------+-------------------+-------------------+-------------------------+-----------+
|    app_id|            app_name|  timestamp_created|  timestamp_updated|author_playtime_at_review|recommended|
+----------+--------------------+-------------------+-------------------+-------------------------+-----------+
|    292030|The Witcher 3: Wi...|2021-01-22 22:00:29|2021-01-22 22:00:29|                   1909.0|       true|
|    292030|The Witcher 3: Wi...|2021-01-22 21:50:30|2021-01-22 21:50:30|                   2674.0|       true|
|    292030|The Witcher 3: Wi...|2021-01-22 21:46:40|2021-01-22 21:46:40|                   1060.0|       true|
|    292030|The Witcher 3: Wi...|2021-01-22 21:32:50|2021-01-22 21:32:50|                   5524.0|       true|
|    292030|The Witcher 3: Wi...|2021-01-22 21:23:47|2021-01-22 21:23:47|                    217.0|       true|
|    292030|The Witcher 3: Wi...|2021-01-22 21:21:04|2021-01-22 21:21:04|                    823.0|     

In [36]:
data_rev = df_clean.select(*col_rev)
data_rev.show()

+----------+--------------------+--------+-----------+
|    app_id|            app_name|language|recommended|
+----------+--------------------+--------+-----------+
|    292030|The Witcher 3: Wi...|schinese|       true|
|    292030|The Witcher 3: Wi...|schinese|       true|
|    292030|The Witcher 3: Wi...|schinese|       true|
|    292030|The Witcher 3: Wi...| english|       true|
|    292030|The Witcher 3: Wi...|schinese|       true|
|    292030|The Witcher 3: Wi...| english|       true|
|    292030|The Witcher 3: Wi...| english|       true|
|    292030|The Witcher 3: Wi...|schinese|       true|
|    292030|The Witcher 3: Wi...| spanish|       true|
|    292030|The Witcher 3: Wi...|schinese|       true|
|    292030|The Witcher 3: Wi...| russian|       true|
|1611375772|          1611375772|       0|       true|
|    292030|The Witcher 3: Wi...| russian|       true|
|    292030|The Witcher 3: Wi...| koreana|       true|
|    292030|The Witcher 3: Wi...|   latam|       true|
|    29203

In [37]:
data_rec = df_clean.select(*col_rec)
data_rec.show()

+----------+--------------------+-----------+
|    app_id|            app_name|recommended|
+----------+--------------------+-----------+
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|1611375772|          1611375772|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...|       true|
|    292030|The Witcher 3: Wi...| 

In [38]:
# Try to read the parquet back
try:
    test_df = spark.read.parquet("/user/tejashree/project/cleaned_steam_reviews.parquet")
    print("Parquet read successful. Number of rows:", test_df.count())
except Exception as e:
    print("Error reading parquet:", str(e))


Parquet read successful. Number of rows: 20595477


### filter data for analysis langauga = english

In [5]:
df_cleaned_parquet = spark.read.parquet("/user/tejashree/project/cleaned_steam_reviews.parquet")


In [7]:
english_df = df_cleaned_parquet.filter(df_cleaned_parquet.language == "english")


In [9]:
english_df.write.mode("overwrite").parquet("/user/tejashree/project/steam_review_english.parquet")


In [11]:
try:
    test_df = spark.read.parquet("/user/tejashree/project/steam_review_english.parquet")
    print("Parquet read successful. Number of rows:", test_df.count())
except Exception as e:
    print("Error reading parquet:", str(e))


Parquet read successful. Number of rows: 7327298


In [13]:
spark.stop()